<a href="https://colab.research.google.com/github/nathalyAlarconT/adk-training-notebooks/blob/main/ADK_Basic_Serper_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An agent that searches for news on the web using Serper.

This agent will have one tool to search in the news using Serper.

# Required Libraries

In [1]:
%%capture
!pip install google-adk
!pip install crewai-tools

# Ingredients

## First Ingredient
Let's get the API KEYs that we need:
- GOOGLE API KEY
Go to: https://aistudio.google.com/apikey

Create a Secret in Google Colab with the following name: GOOGLE_API_KEY

## Second Ingredient
We are going to use SERPER to perform searches in Google
- SERPER_API_KEY Go to: https://serper.dev/api-key

Create a Secret in Google Colab with the following name: SERPER_API_KEY

# Let's begin

In [2]:
import os
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.crewai_tool import CrewaiTool
from google.genai import types
from crewai_tools import SerperDevTool

In [4]:
from google.colab import userdata

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Constants
APP_NAME = "news_app"
USER_ID = "user1"
SESSION_ID = "sess_1"



In [19]:
# Let's define the model to use
LLM_MODEL = "gemini-2.0-flash-lite"

In [20]:

# Ensure SERPER_API_KEY is set in your environment

SERVER_API_KEY = userdata.get('SERPER_API_KEY')

os.environ['SERPER_API_KEY'] = SERVER_API_KEY

serper_tool_instance = SerperDevTool(
    n_results=3,
    save_file=False,

)


In [21]:
adk_serper_tool = CrewaiTool(
    name="InternetNewsSearch",
    description="This tool performs targeted searches on the internet to find recent news articles. Use it when you need to gather up-to-date information on current events or specific topics that are likely to be covered in news publications.",
    tool=serper_tool_instance
)

In [22]:
serper_agent = Agent(
    name="internet_research_agent", # More descriptive name
    model=LLM_MODEL,
    description="An expert agent designed to find answers to user questions by performing **targeted searches across the internet** using the Serper API.",
    instruction="My primary function is to **accurately answer your questions by searching the internet for relevant information**. Provide clear and concise queries for the best results.",
    tools=[adk_serper_tool]
)

In [23]:
# Session and Runner
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")


runner = Runner(agent=serper_agent, app_name=APP_NAME, session_service=session_service)


Session created: App='news_app', User='user1', Session='sess_1'


In [24]:
# Agent Interaction
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)



In [25]:
call_agent("what's are the latest news about AI Agents?")

Using Tool: Search the internet with Serper
Agent Response:  Here's the latest news about AI agents:

*   A new AI agent has "shocked the market" (source: YouTube).
*   China is experiencing an AI agent boom, with a new general AI agent showing promise despite some initial issues (source: technologyreview.com).


In [26]:
call_agent("what's are the latest news about Bolivia?")

Using Tool: Search the internet with Serper
Agent Response:  Here's the latest news about Bolivia:

*   A bus crash in Bolivia killed at least 37 people (source: BBC News).
*   Five people survived 36 hours in a swamp after a plane crash (source: BBC News).
*   Incumbent Luis Arce quit Bolivia's presidential race amid slumping support (source: Al Jazeera).
*   Venezuela defeated Bolivia 2-0 in a soccer match (source: AP News).
